In [1]:
#On Colab only
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [92]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv
import os

In [93]:
# Load the .env file
load_dotenv()

# Get the token from .env
token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    # Login to Hugging Face
    login(token=token)
    print("Successfully logged in to Hugging Face.")
else:
    print("Token not found. Please check your .env file.")

Successfully logged in to Hugging Face.


In [94]:
# Load the Amazon Reviews 2023 dataset
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_meta_All_Beauty")

# View the dataset structure
print(dataset)

DatasetDict({
    full: Dataset({
        features: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together', 'subtitle', 'author'],
        num_rows: 112590
    })
})


In [95]:
import pandas as pd

In [96]:
# Access the 'full' split
full_data = dataset['full']

# View the first few records
data = full_data[:5]
pd.DataFrame(data)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [13]:
# Take a random 20% sample
subset = full_data.train_test_split(test_size=0.2)['test']

# Drop columns 'description', 'features', and 'categories' if they exist
columns_to_drop = ["description", "features", "categories"]
subset = subset.remove_columns([col for col in columns_to_drop if col in subset.column_names])

# Verify the size of the subset
print(f"Original size: {len(full_data)}, Subset size: {len(subset)}")
subset.save_to_disk("amazon_reviews_subset")

Original size: 112590, Subset size: 22518


Saving the dataset (0/1 shards):   0%|          | 0/22518 [00:00<?, ? examples/s]

In [ ]:
# Login to Hugging Face. Not required if HF token provided in .env file. 
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import HfApi
from datasets import Dataset

# Create a new dataset repository
repo_id = "ashishkumarsahani/amazon-beauty-data-subset"  # Replace `your_username` with your Hugging Face username
api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

# Push the dataset to Hugging Face
subset.push_to_hub(repo_id=repo_id, token=True)

print(f"Dataset uploaded to: https://huggingface.co/datasets/{repo_id}")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Dataset uploaded to: https://huggingface.co/datasets/ashishkumarsahani/amazon-appliances-data-subset
